## Named Entity Recognition

In [1]:
import spacy
from nltk import sent_tokenize

In [2]:
!python -m spacy download en_core_web_trf

     ---------------------------------------- 0.0/457.4 MB ? eta -:--:--
     ---------------------------------------- 1.0/457.4 MB 6.3 MB/s eta 0:01:13
     ---------------------------------------- 1.8/457.4 MB 4.8 MB/s eta 0:01:36
     ---------------------------------------- 2.6/457.4 MB 4.4 MB/s eta 0:01:43
     ---------------------------------------- 3.4/457.4 MB 4.4 MB/s eta 0:01:44
     ---------------------------------------- 4.2/457.4 MB 4.3 MB/s eta 0:01:47
     ---------------------------------------- 5.0/457.4 MB 4.2 MB/s eta 0:01:48
      --------------------------------------- 5.8/457.4 MB 4.2 MB/s eta 0:01:48
      --------------------------------------- 6.6/457.4 MB 4.2 MB/s eta 0:01:49
      --------------------------------------- 7.6/457.4 MB 4.1 MB/s eta 0:01:50
      --------------------------------------- 8.4/457.4 MB 4.1 MB/s eta 0:01:50
      --------------------------------------- 9.2/457.4 MB 4.1 MB/s eta 0:01:51
      -------------------------------------- 10

## Load Model

In [2]:
def load_model():
    nlp = spacy.load("en_core_web_trf")
    return nlp

In [3]:

nlp_model = load_model()

e:\Users\prath\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Users\prath\anaconda3\envs\venv\lib\site-packages\thinc\shims\pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_global

## load Dataset

In [4]:
import os 
import sys
import pathlib
folder_path = pathlib.Path().parent.resolve()
sys.path.append(os.path.join(folder_path, '../'))
from utils import load_subtitles_dataset

In [5]:
dataset_path = "../data/Subtitles/"
df = load_subtitles_dataset(dataset_path)

In [6]:
df.head()

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [7]:
sample_script = df.iloc[0]['script']
sample_script

'A long time ago, a powerful demon fox appeared with nine tails.\n With its powerful tails,\n it could smash mountains and create tidal waves.\n A band of Ninjas rose to defend their village from attack.\n We have to wait until the Fourth Hokage gets here!\n We can\'t let it get any closer to our village!\n One great Ninja was able to imprison the monster,\n but died in the process.\n This Ninja was known asΓÇª the Fourth Hokage.\n Naruto!\n Why did you do such a thing?!\n You\'re really gonna get it this time!\n I don\'t care!\n You know your problem?\n You can\'t do the things I do!\n Only I can do this!\n I\'m better than all of you! Believe it!\n There\'s a problem, sir!\n Lord Hokage!\n What is it?\n Did that Naruto do something again?\n Yes. He climbed onto the Mountainside ImagesΓÇª\n And he vandalized and graffitied all over them!\n Wait!\n Ha haΓÇª\n Why should I?\n Hey, Naruto!\n How did you suddenly get here, lruka Sensei?\n The question is what are you doing here when you s

In [8]:
sentences = sent_tokenize(sample_script)

In [9]:
sentences = sentences[60:90]

In [10]:
sentence = ".".join(sentences)

In [11]:
sentence

"Don't you know who the Hokage leaders are?.Of course, I do!.I know they earned the title Lord Hokage\n because they were the best Ninja of their time, right?.Especially the Fourth Hokage was a hero\n who saved the village from the nine-tail demon fox..Then why did you do that?.Because I'll become a Hokage myself..And I'll be the greatest Hokage of all time!.So that everyone will finally learn to accept me!.By the way, Sensei, I have a favor to ask..You want another bowl?.MmmmΓÇªNoΓÇª\n Can I borrow that Leaf headband for a while?.This?.No no!.This is worn only by those who have graduated from Ninja Academy..Tomorrow, you willΓÇª\n You're so mean!.So that's why you took off your gogglesΓÇª\n Humph... One more bowl please!.We are now about to begin the graduation test..When your name is called, proceed to the next classroom..The test is on the Clone Jutsu..Oh noΓÇª\n Of all theΓÇª!.That is my weakest Jutsu!.But stillΓÇª I will do it no matter what!.Clone Jutsu!.Disqualified!.Iruka Sense

## Run Model

In [12]:
doc = nlp_model(sentence)

e:\Users\prath\anaconda3\envs\venv\lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [13]:
doc.ents

(Ninja,
 Fourth,
 nine,
 Leaf,
 Ninja Academy,
 One,
 the Clone Jutsu,
 three,
 Naruto,
 one)

In [14]:
for entity in doc.ents:
    print(entity, entity.label_)

Ninja NORP
Fourth ORDINAL
nine CARDINAL
Leaf PRODUCT
Ninja Academy ORG
One CARDINAL
the Clone Jutsu PRODUCT
three CARDINAL
Naruto PERSON
one CARDINAL


In [15]:
def get_ners_inference(script):
    script_sentences = sent_tokenize(script)

    ner_output = []

    for sentence in script_sentences:
        doc = nlp_model(sentence)
        ners = set()
        for entity in doc.ents:
            if entity.label_ =="PERSON":
                full_name = entity.text
                first_name = entity.text.split(" ")[0]
                first_name = first_name.strip()
                ners.add(first_name)
        ner_output.append(ners)

    return ner_output

In [16]:
df = df.head(10)

In [17]:
df

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."
5,6,"C'mon!\n Running like a fugitive,\n Being chas..."
6,7,"C'mon!\n Running like a fugitive,\n Being chas..."
7,8,"C'mon!\n Running like a fugitive,\n Being chas..."
8,9,"C'mon!\n Running like a fugitive,\n Being chas..."
9,12,"C'mon!\n Running like a fugitive,\n Being chas..."


In [18]:
df['ners'] = df['script'].apply(get_ners_inference)

e:\Users\prath\anaconda3\envs\venv\lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [19]:
df

,episode,script,ners
0,1,"A long time ago, a powerful demon fox appeared...","[{}, {}, {}, {}, {}, {}, {}, {Naruto}, {}, {},..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {Konohama..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {Sasuke, Sakura}, {}, {Konohamaru..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {Naruto}, {}, {}, {Iruka}, {}, {N..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {..."
5,6,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {Sasuke}, {Sakura}, {Naruto}, {}, {Na..."
6,7,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {..."
7,8,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {}, {}, {}, {Sasuke}, {}, {},..."
8,9,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {..."
9,12,"C'mon!\n Running like a fugitive,\n Being chas...","[{}, {}, {}, {}, {Zabuza}, {}, {}, {}, {Naruto..."


## Character Network

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network

In [21]:
def generate_character_network(df):

    windows=10
    entity_relationship = []

    for row in df['ners']:
        previous_entities_in_window = []

        for sentence in row:
            previous_entities_in_window.append(list(sentence))
            previous_entities_in_window = previous_entities_in_window[-windows:]

            # Flatten 2D List into 1D List
            previous_entities_flattened = sum(previous_entities_in_window, [])

            for entity in sentence:
                for entity_in_window in previous_entities_flattened:
                    if entity != entity_in_window:
                        entity_relationship.append(sorted([entity, entity_in_window]))
    
    relationship_df = pd.DataFrame({'value': entity_relationship})
    relationship_df['source'] = relationship_df['value'].apply(lambda x: x[0])
    relationship_df['target'] = relationship_df['value'].apply(lambda x: x[1])
    relationship_df = relationship_df.groupby(['source', 'target']).count().reset_index()
    relationship_df = relationship_df.sort_values('value', ascending=False)

    return relationship_df

In [22]:
relationship_df = generate_character_network(df)

In [23]:
relationship_df

,source,target,value
102,Naruto,Sasuke,80
59,Iruka,Naruto,34
124,Sakura,Sasuke,33
96,Mizuki,Naruto,26
42,Hokage,Naruto,17
...,...,...,...
52,Ino,Naruto,1
121,Rogue,the,1
54,Ino,Sasuke,1
118,Rogue,The,1


In [24]:

relationship_df = relationship_df.sort_values('value', ascending=False)
relationship_df = relationship_df.head(200)

In [26]:
G = nx.from_pandas_edgelist(
    relationship_df, 
    source='source', 
    target='target', 
    edge_attr='value',
    create_using=nx.Graph()
)

net = Network(notebook=True, width="1000px", height="700px", bgcolor="#222222", font_color="white", cdn_resources="remote")
node_degree = dict(G.degree)

nx.set_node_attributes(G, node_degree, 'size')
net.from_nx(G)
net.show("naruto.html")

naruto.html
